In [166]:
# pip install webdriver-manager


# Speak function

In [176]:
import pyttsx3
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voices', voices[0].id)
engine.setProperty('rate', 150)

def speak(text):
    print("")
    print(f'Jarvis : {text}.')
    print("")
    engine.say(text)
    engine.runAndWait()


# Listen 

In [165]:
import speech_recognition as sr
from googletrans import Translator #pip install googletrans==3.1.0a0

# 1 - Listen : Hindi or English

def Listen():

    r = sr.Recognizer()

    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source,0,8) # Listening Mode.....
    
    try:
        print("Recognizing...")
        query = r.recognize_google(audio,language="en-in")

    except:
        return ""
    
    query = str(query).lower()
    return query

# 2 - Translation

def TranslationHinToEng(Text):
    line = str(Text)
    translate = Translator()
    result = translate.translate(line)
    data = result.text
    print(f"You : {data}.")
    return data

# 3 - Connect

def MicExecution():
    query = Listen()
    data = TranslationHinToEng(query)
    return data
MicExecution()


Listening...
Recognizing...
You : .


''

# Brain

In [177]:
import torch.nn as nn #pip install torch

class NeuralNet(nn.Module):

    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.l1 = nn.Linear(input_size,hidden_size)
        self.l2 = nn.Linear(hidden_size,hidden_size)
        self.l3 = nn.Linear(hidden_size,num_classes)
        self.relu = nn.ReLU()

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

# Tokenization

In [178]:
import numpy as np #pip install numpy
import nltk #pip install nltk
from nltk.stem.porter import PorterStemmer

Stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return Stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence,words):
    sentence_word = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words),dtype=np.float32)

    for idx , w in enumerate(words):
        if w in sentence_word:
            bag[idx] = 1

    return bag


# Train

In [181]:
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
# from NeuralNetwork import bag_of_words, tokenize, stem
# from Brain import NeuralNet
# from Speak import Speak
# from Listen import TexttoSpeech



with open('intent.json', 'r') as f:
    intents = json.load(f)


all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)

    for pattern in intent['patterns']:
       # print(pattern)
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = [',','?','/','.','!']
all_words = [stem(w) for w in all_words if w not in ignore_words]        

all_words = sorted(set(all_words))

tags = sorted(set(tags))
x_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)


x_train = np.array(x_train)
y_train = np.array(y_train)

num_epochs = 20000
batch_size = 10
learning_rate = 0.00001
input_size = len(x_train[0])
hidden_size = 10
output_size = len(tags)
    

speak("Jarvis is in the training mode...")


class Chatdataset(Dataset):
    
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]  

    def __len__(self):
        return self.n_samples

dataset = Chatdataset()

train_loader = DataLoader(dataset = dataset,
                          batch_size = batch_size,
                          shuffle = True,
                          num_workers = 0)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)

        labels = labels.to(dtype=torch.long).to(device)
        outputs = model(words)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100==0:
        print(f"Epoch [{epoch +1}/{num_epochs}], loss : {loss.item():.4f}") 

speak(f'Final Loss : {loss.item():.4f}')
speak('I guess the loss is pretty much less...')
speak("No need to train me again Mr. Tushar!")


speak("Training has been completed")
data = {
    'model_state':model.state_dict(),
    'input_size':input_size,
    'hidden_size':hidden_size,
    'output_size':output_size,
    'all_words':all_words,
    'tags':tags
}

FILE = "TrainData.pth"
torch.save(data, FILE)

speak(f"Training completed, File saved to {FILE}")




Jarvis : Jarvis is in the training mode....

Epoch [100/20000], loss : 4.3885
Epoch [200/20000], loss : 4.4717
Epoch [300/20000], loss : 4.3788
Epoch [400/20000], loss : 4.3835
Epoch [500/20000], loss : 4.2571
Epoch [600/20000], loss : 4.2310
Epoch [700/20000], loss : 4.3520
Epoch [800/20000], loss : 4.2443
Epoch [900/20000], loss : 4.1848
Epoch [1000/20000], loss : 4.1688
Epoch [1100/20000], loss : 4.1099
Epoch [1200/20000], loss : 4.0709
Epoch [1300/20000], loss : 4.1286
Epoch [1400/20000], loss : 3.8571
Epoch [1500/20000], loss : 3.7715
Epoch [1600/20000], loss : 4.0239
Epoch [1700/20000], loss : 3.8988
Epoch [1800/20000], loss : 3.5524
Epoch [1900/20000], loss : 3.7141
Epoch [2000/20000], loss : 3.2970
Epoch [2100/20000], loss : 3.0056
Epoch [2200/20000], loss : 3.3802
Epoch [2300/20000], loss : 3.1509
Epoch [2400/20000], loss : 2.9342
Epoch [2500/20000], loss : 2.8776
Epoch [2600/20000], loss : 3.1078
Epoch [2700/20000], loss : 2.7177
Epoch [2800/20000], loss : 2.4333
Epoch [2900

# Load the model for prediction

In [192]:
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intent.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = 'TrainData.pth'
data = torch.load(FILE)    

input_size = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size)
model.load_state_dict(model_state)
model.eval()

Name = "Jarvis"


def Main():
    while 1:
        query = input("Enter the query : ")
        if 'stop' or 'get out' or 'leave' or 'good bye' in query:
            os.exi
            

        query = tokenize(query)
        X = bag_of_words(query, all_words) 
        X = X.reshape(1, X.shape[0]) 
        X = torch.from_numpy(X).to(device)

        output = model(X)
        _, predicted = torch.max(output, dim=1)
        tag = tags[predicted.item()]

        probs = torch.softmax(output, dim=1) 
        prob = probs[0][predicted.item()]

        if prob.item() > 0.75:
            for intent in intents['intents']:
                if tag == intent['tag']:
                    reply = random.choice(intent['responses'])
                    
                    speak(reply)

Enter the query : hello

Jarvis : How Are You Sir..



In [197]:
reply
